In [1]:
import os, sys
import numpy as np
# path to access c++ files
installation_path = os.getenv("INSTALL_PATH")
sys.path.append(installation_path)

In [2]:
from cunqa import getQPUs

qpus  = getQPUs()

for q in qpus:
    print(f"QPU {q.id}, backend: {q.backend.name}, simulator: {q.backend.simulator}, version: {q.backend.version}.")


	info: Logger created.
	debug: File accessed correctly.
	debug: Object for QPU 0 created correctly.
	debug: Object for QPU 1 created correctly.
	debug: Object for QPU 2 created correctly.
	debug: 3 QPU objects were created.
QPU 0, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 1, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.
QPU 2, backend: BasicMunich, simulator: MunichSimulator, version: 0.0.1.


# Paralelization for gradient-free optimizers: Differential Evolution

_Introduction and explanation_

We recover the variational circuit used before:

In [3]:
from qiskit import QuantumCircuit
from qiskit.circuit import Parameter

def hardware_efficient_ansatz(num_qubits, num_layers):
    qc = QuantumCircuit(num_qubits)
    param_idx = 0
    for _ in range(num_layers):
        for qubit in range(num_qubits):
            phi = Parameter(f'phi_{param_idx}_{qubit}')
            lam = Parameter(f'lam_{param_idx}_{qubit}')
            qc.ry(phi, qubit)
            qc.rz(lam, qubit)
        param_idx += 1
        for qubit in range(num_qubits - 1):
            qc.cx(qubit, qubit + 1)
    qc.measure_all()
    return qc

In [4]:
def target_distribution(num_qubits):
    # Define a normal distribution over the states
    num_states = 2 ** num_qubits
    states = np.arange(num_states)
    mean = num_states / 2
    std_dev = num_states / 4
    target_probs = norm.pdf(states, mean, std_dev)
    target_probs /= target_probs.sum()  # Normalize to make it a valid probability distribution
    target_dist = {format(i, f'0{num_qubits}b'): target_probs[i] for i in range(num_states)}
    return target_dist

import pandas as pd
from scipy.stats import entropy, norm

def KL_divergence(counts, n_shots, target_dist):
    # Convert counts to probabilities
    pdf = pd.DataFrame.from_dict(counts, orient="index").reset_index()
    pdf.rename(columns={"index": "state", 0: "counts"}, inplace=True)
    pdf["probability"] = pdf["counts"] / n_shots
    
    # Create a dictionary for the obtained distribution
    obtained_dist = pdf.set_index("state")["probability"].to_dict()
    
    # Ensure all states are present in the obtained distribution
    for state in target_dist:
        if state not in obtained_dist:
            obtained_dist[state] = 0.0
    
    # Convert distributions to lists for KL divergence calculation
    target_probs = [target_dist[state] for state in sorted(target_dist)]
    obtained_probs = [obtained_dist[state] for state in sorted(obtained_dist)]
    
    # Calculate KL divergence
    kl_divergence = entropy(obtained_probs, target_probs)
    
    return kl_divergence
    

In [5]:
num_qubits = 6

num_layers = 3

n_shots = 100000

target_dist = target_distribution(num_qubits)

In [6]:
def cost_function(result):
    
    global target_dist
    
    counts = result.get_counts()
    
    return KL_divergence(counts, n_shots, target_dist)

In [7]:
ansatz = hardware_efficient_ansatz(num_qubits, num_layers)

num_parameters = ansatz.num_parameters; print(num_parameters)

initial_parameters = np.zeros(num_parameters)

36


In [8]:
init_qjobs = []
init_params = np.zeros(num_parameters)
for q in qpus:
    init_qjobs.append(q.run(ansatz.assign_parameters(init_params), transpile=False, shots=n_shots))

from cunqa import QJobMapper
mapper = QJobMapper(init_qjobs)

	debug: A QuantumCircuit was provided.
	debug: Translating to QASM2 for MunichSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188}, "instructions":"OPENQASM 2.0;\ninclude \"qelib1.inc\";\nqreg q[6];\ncreg meas[6];\nry(0) q[0];\nrz(0) q[0];\nry(0) q[1];\nrz(0) q[1];\nry(0) q[2];\nrz(0) q[2];\nry(0) q[3];\nrz(0) q[3];\nry(0) q[4];\nrz(0) q[4];\nry(0) q[5];\nrz(0) q[5];\ncx q[0],q[1];\ncx q[1],q[2];\ncx q[2],q[3];\ncx q[3],q[4];\ncx q[4],q[5];\nry(0) q[0];\nrz(0) q[0];\nry(0) q[1];\nrz(0) q[1];\nry(0) q[2];\nrz(0) q[2];\nry(0) q[3];\nrz(0) q[3];\nry(0) q[4];\nrz(0) q[4];\nry(0) q[5];\nrz(0) q[5];\ncx q[0],q[1];\ncx q[1],q[2];\ncx q[2],q[3];\ncx q[3],q[4];\ncx q[4],q[5];\nry(0) q[0];\nrz(0) q[0];\nry(0) q[1];\nrz(0) q[1];\nry(0) q[2];\nrz(0) q[2];\nry(0) q[3];\nrz(0) q[3];\nry(0) q[4];\nrz(0) q[4];\nry(0) q[5];\nrz(0) q[5];\ncx q[0],q[1];\ncx q[1],q[2];\ncx q[2],q[3];\ncx q[3],q[4];\ncx q[4],q[5];\nbarrier q[0],q[1

In [9]:
pop=[]
total_pop=1*num_parameters
for j in range(total_pop):
    initial_point=np.random.uniform(-np.pi, np.pi, num_parameters)
    pop.append(initial_point)

bounds=[]
for i in range(0,num_parameters):
    bounds.append((-np.pi,np.pi))

print("Bounds:", len(bounds))
print("Initial population:", len(pop))

best_individual = []

def cb(xk,convergence=1e-8):
     best_individual.append(xk)

from scipy.optimize import differential_evolution
import time

tick = time.time()
result = differential_evolution(cost_function, bounds, maxiter=10, disp=True, workers=mapper, strategy='best1bin', init=pop, polish = False, callback=cb)
tack = time.time()
print(result)

energies = mapper(cost_function, best_individual)



print("Time:", tack-tick)

Bounds: 36
Initial population: 36
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb362214c0> in QPU 0...
	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb3620ac10> in QPU 1...
	debug: Sending parameters to QPU 1.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb700345b0> in QPU 2...
	debug: Sending parameters to QPU 2.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb362214c0> in QPU 0...
	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb3620ac10> in QPU 1...
	debug: Sending parameters to QPU 1.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb700345b0> in QPU 2...
	debug: Sending parameters to QPU 2.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb362214c0> in QPU 0...
	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb3620ac10> in QPU 1...
	de

/opt/cesga/qmio/hpc/software/Compiler/gcc/12.3.0/scipy/1.13.0-python-3.9.9/lib/python3.9/site-packages/scipy/optimize/_differentialevolution.py:487: UserWarning: differential_evolution: the 'workers' keyword has overridden updating='immediate' to updating='deferred'
  with DifferentialEvolutionSolver(func, bounds, args=args,


	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb3620ac10> in QPU 1...
	debug: Sending parameters to QPU 1.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb700345b0> in QPU 2...
	debug: Sending parameters to QPU 2.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb362214c0> in QPU 0...
	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb3620ac10> in QPU 1...
	debug: Sending parameters to QPU 1.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb700345b0> in QPU 2...
	debug: Sending parameters to QPU 2.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb362214c0> in QPU 0...
	debug: Sending parameters to QPU 0.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb3620ac10> in QPU 1...
	debug: Sending parameters to QPU 1.
	debug: Uptading params for QJob <cunqa.qjob.QJob object at 0x14bb700345b0> in QPU 2...


QJobError: 

In [10]:
dir(result)

['fun',
 'message',
 'nfev',
 'nit',
 'population',
 'population_energies',
 'success',
 'x']

In [11]:
result.population_energies

array([0.32719802, 0.42013288, 0.40499936, 0.42823886, 0.44627918,
       0.33285421, 0.57141561, 0.44878708, 0.56298556, 0.37689749,
       0.42306269, 0.55143333, 0.35464565, 0.55703338, 0.47138315,
       0.58365131, 0.42436711, 0.52933254, 0.45537101, 0.45975614,
       0.43082161, 0.42967797, 0.65314656, 0.51136848, 0.32999285,
       0.54880022, 0.50273282, 0.33058816, 0.41816775, 0.34620732,
       0.36608831, 0.42013288, 0.40499936, 0.42823886, 0.44627918,
       0.33285421])

In [12]:
result.x

array([ 1.51129225,  1.54685148,  0.64145606, -0.69808512, -1.29832276,
        0.80559764, -0.69420306, -2.08385609, -0.86399622, -0.94787747,
       -0.86790541, -0.80229439,  2.76004886, -1.9887602 , -0.22065667,
        1.13869392, -0.94683632,  1.39698509, -0.20020418,  2.3966701 ,
        1.26902094, -1.87304704, -1.4481734 , -2.32071898,  0.25574782,
       -2.32655588, -1.34577899,  2.4670583 , -1.32263484, -2.01088552,
       -1.75915757, -2.08058682,  1.78410493, -0.8515121 , -0.60741441,
        3.06997263])

In [13]:
circ = ansatz.assign_parameters(result.x)

In [33]:
energies

[0.727562237626633,
 0.5375982776510955,
 0.8481303502149584,
 0.7380450645063268,
 0.7884340047902858,
 0.6218309345052601,
 0.9141965358010642,
 0.9069933133442408,
 0.9017151546155266,
 0.908658796430865,
 0.9044093999901259,
 0.9115332860362739,
 0.9047464179405276,
 0.9039963032741348,
 0.9050659607038982,
 0.9065477748138756,
 0.9043032477440331,
 0.8997411709938532,
 0.9063042647608612,
 0.9087510880924539,
 0.9028235939702883,
 0.9028993728877099,
 0.9086315120488707,
 0.9060376759219009,
 0.9052891321963904,
 0.909549649983757,
 0.9078178329831255,
 0.9078002052833902,
 0.23625076793234898,
 0.23627627817923913,
 0.727562237626633,
 0.5375982776510955,
 0.8481303502149584,
 0.7380450645063268,
 0.7884340047902858,
 0.6218309345052601,
 0.9141965358010642,
 0.9069933133442408,
 0.9017151546155266,
 0.908658796430865,
 0.9044093999901259,
 0.9115332860362739,
 0.9047464179405276,
 0.9039963032741348,
 0.9050659607038982,
 0.9065477748138756,
 0.9043032477440331,
 0.8997411709938

In [15]:
for i in range(10):
    print(qpus[0].backend.name)
    counts = qpus[0].run(circ, shots = n_shots, seed_simulator = 12).result().get_counts()
    print(counts)

BasicAer
	debug: A QuantumCircuit was provided.
	debug: Translating to dict for AerSimulator...
	debug: QJob created.
	debug:  {"config":{"shots": 100000, "method": "statevector", "memory_slots": 6, "seed": 188, "seed_simulator": 12}, "instructions":[{"name": "ry", "qubits": [0], "params": [-0.20020418383969812]}, {"name": "rz", "qubits": [0], "params": [1.5112922466895622]}, {"name": "ry", "qubits": [1], "params": [2.3966701023101122]}, {"name": "rz", "qubits": [1], "params": [1.5468514776878188]}, {"name": "ry", "qubits": [2], "params": [1.2690209403077888]}, {"name": "rz", "qubits": [2], "params": [0.6414560606776722]}, {"name": "ry", "qubits": [3], "params": [-1.8730470446960625]}, {"name": "rz", "qubits": [3], "params": [-0.6980851195077713]}, {"name": "ry", "qubits": [4], "params": [-1.448173404100087]}, {"name": "rz", "qubits": [4], "params": [-1.2983227641628658]}, {"name": "ry", "qubits": [5], "params": [-2.3207189838897624]}, {"name": "rz", "qubits": [5], "params": [0.8055976

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 0.
	debug: Results correctly loaded.
{'000000': 1941, '000001': 4452, '010000': 278, '010001': 3480, '010010': 494, '010011': 835, '010100': 5337, '010101': 240, '010110': 1318, '010111': 349, '011000': 20, '011001': 32, '011010': 251, '011011': 382, '011100': 1409, '011101': 44, '011110': 631, '011111': 790, '000010': 3861, '100000': 867, '100001': 548, '100010': 458, '100011': 1509, '100100': 329, '100101': 410, '100110': 32, '100111': 9, '101000': 384, '101001': 2453, '101010': 145, '101011': 5294, '101100': 1292, '101101': 1440, '101110': 3775, '101111': 117, '000011': 90, '110000': 1385, '110001': 3897, '110010': 5335, '110011': 21, '110100': 457, '110101': 414, '110110': 211, '110111': 377, '111000': 85, '111001': 4300, '111010': 6129, '111011': 1004, '111100': 344, '111101': 4169, '111110': 2828, '111111': 1627, '000100': 779, '000101': 8151, '000110': 3233, '000111': 190, '001000': 288, '001001': 357, '001010': 243, '00101

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 0.
	debug: Results correctly loaded.
{'000000': 1941, '000001': 4452, '010000': 278, '010001': 3480, '010010': 494, '010011': 835, '010100': 5337, '010101': 240, '010110': 1318, '010111': 349, '011000': 20, '011001': 32, '011010': 251, '011011': 382, '011100': 1409, '011101': 44, '011110': 631, '011111': 790, '000010': 3861, '100000': 867, '100001': 548, '100010': 458, '100011': 1509, '100100': 329, '100101': 410, '100110': 32, '100111': 9, '101000': 384, '101001': 2453, '101010': 145, '101011': 5294, '101100': 1292, '101101': 1440, '101110': 3775, '101111': 117, '000011': 90, '110000': 1385, '110001': 3897, '110010': 5335, '110011': 21, '110100': 457, '110101': 414, '110110': 211, '110111': 377, '111000': 85, '111001': 4300, '111010': 6129, '111011': 1004, '111100': 344, '111101': 4169, '111110': 2828, '111111': 1627, '000100': 779, '000101': 8151, '000110': 3233, '000111': 190, '001000': 288, '001001': 357, '001010': 243, '00101

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 0.
	debug: Results correctly loaded.
{'000000': 1941, '000001': 4452, '010000': 278, '010001': 3480, '010010': 494, '010011': 835, '010100': 5337, '010101': 240, '010110': 1318, '010111': 349, '011000': 20, '011001': 32, '011010': 251, '011011': 382, '011100': 1409, '011101': 44, '011110': 631, '011111': 790, '000010': 3861, '100000': 867, '100001': 548, '100010': 458, '100011': 1509, '100100': 329, '100101': 410, '100110': 32, '100111': 9, '101000': 384, '101001': 2453, '101010': 145, '101011': 5294, '101100': 1292, '101101': 1440, '101110': 3775, '101111': 117, '000011': 90, '110000': 1385, '110001': 3897, '110010': 5335, '110011': 21, '110100': 457, '110101': 414, '110110': 211, '110111': 377, '111000': 85, '111001': 4300, '111010': 6129, '111011': 1004, '111100': 344, '111101': 4169, '111110': 2828, '111111': 1627, '000100': 779, '000101': 8151, '000110': 3233, '000111': 190, '001000': 288, '001001': 357, '001010': 243, '00101

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 0.
	debug: Results correctly loaded.
{'000000': 1941, '000001': 4452, '010000': 278, '010001': 3480, '010010': 494, '010011': 835, '010100': 5337, '010101': 240, '010110': 1318, '010111': 349, '011000': 20, '011001': 32, '011010': 251, '011011': 382, '011100': 1409, '011101': 44, '011110': 631, '011111': 790, '000010': 3861, '100000': 867, '100001': 548, '100010': 458, '100011': 1509, '100100': 329, '100101': 410, '100110': 32, '100111': 9, '101000': 384, '101001': 2453, '101010': 145, '101011': 5294, '101100': 1292, '101101': 1440, '101110': 3775, '101111': 117, '000011': 90, '110000': 1385, '110001': 3897, '110010': 5335, '110011': 21, '110100': 457, '110101': 414, '110110': 211, '110111': 377, '111000': 85, '111001': 4300, '111010': 6129, '111011': 1004, '111100': 344, '111101': 4169, '111110': 2828, '111111': 1627, '000100': 779, '000101': 8151, '000110': 3233, '000111': 190, '001000': 288, '001001': 357, '001010': 243, '00101

	debug: Circuit was sent.
	debug: Qjob submitted to QPU 0.
	debug: Results correctly loaded.
{'000000': 1941, '000001': 4452, '010000': 278, '010001': 3480, '010010': 494, '010011': 835, '010100': 5337, '010101': 240, '010110': 1318, '010111': 349, '011000': 20, '011001': 32, '011010': 251, '011011': 382, '011100': 1409, '011101': 44, '011110': 631, '011111': 790, '000010': 3861, '100000': 867, '100001': 548, '100010': 458, '100011': 1509, '100100': 329, '100101': 410, '100110': 32, '100111': 9, '101000': 384, '101001': 2453, '101010': 145, '101011': 5294, '101100': 1292, '101101': 1440, '101110': 3775, '101111': 117, '000011': 90, '110000': 1385, '110001': 3897, '110010': 5335, '110011': 21, '110100': 457, '110101': 414, '110110': 211, '110111': 377, '111000': 85, '111001': 4300, '111010': 6129, '111011': 1004, '111100': 344, '111101': 4169, '111110': 2828, '111111': 1627, '000100': 779, '000101': 8151, '000110': 3233, '000111': 190, '001000': 288, '001001': 357, '001010': 243, '00101

In [22]:
from qiskit import QuantumCircuit
qc = QuantumCircuit(2)
qc.rx(1.77, 0)
qc.measure_all()

for i in range(10):
    print(qpus[0].backend.name)
    counts = qpus[0].run(qc, shots = n_shots, seed = 8).result().get_counts()
    print(counts)

BasicAer
{'00': 99, '01': 1034, '10': 1039, '11': 826}
BasicAer
{'00': 73, '01': 996, '10': 1024, '11': 821}
BasicAer
{'00': 104, '01': 1001, '10': 958, '11': 868}
BasicAer
{'00': 82, '01': 1056, '10': 970, '11': 742}
BasicAer
{'00': 79, '01': 1066, '10': 1001, '11': 796}
BasicAer
{'00': 93, '01': 1026, '10': 977, '11': 768}
BasicAer
{'00': 97, '01': 989, '10': 985, '11': 893}
BasicAer
{'00': 83, '01': 1004, '10': 973, '11': 801}
BasicAer
{'00': 87, '01': 975, '10': 930, '11': 848}
BasicAer
{'00': 79, '01': 988, '10': 1028, '11': 847}


In [18]:
import matplotlib.pyplot as plt
plt.clf()
plt.plot(np.linspace(0, result.nit, result.nit), energies, label="Optimization path (run())")
upper_bound = result.nit
plt.plot(np.linspace(0, upper_bound, upper_bound), np.zeros(upper_bound), "--", label="Target cost")
plt.xlabel("Step"); plt.ylabel("Cost"); plt.legend(loc="upper right"); plt.title(f"n = {num_qubits}, l = {num_layers}, # params = {num_parameters}")
plt.grid(True)
plt.show()
plt.savefig(f"optimization_de_n_{num_qubits}_p_{num_parameters}.png", dpi=200)

In [19]:
lista = [-1.8717609475089807, 2.919760044838526, -1.4124298801537238, 0.9161765699704417, 2.562883105085934, 0.5667637571988291, -2.5725059694192596, 1.235824416778379, -2.9138794541998276, 1.6527400030504462, -2.3482597419936413, 1.9993961466576764, 1.5566463178936045, 0.12987821063892105, 1.0363964446556453, -0.032835304283010125, 0.7800599218456872, 0.6136567977176869, -2.388218620053299, 0.3381876004849084, 2.5001076969431364, 2.498723705381258, 1.158219355120325, -3.140265916264401, -0.6061889006504347, -1.7942180414387496, 2.963679771582688, 2.465252827878361, -2.344759126455823, -1.00580332790408, 1.9092771321163728, -1.6788882738971813, 1.6132266184995945, -1.1472345524117762, -2.119135658127555, 0.5194481651687578]

In [22]:
len([lista])

1

# Paralelization of expectation value terms

In [25]:
# TODO

# Paralelization for gradient optimizers

In [26]:
# TODO